# 🙋🏻‍♂️Шарипов Саит
# 📊 Тестовое задание

## 📄Описание задачи:
Подкинули монету $N$ раз. Кол-во случаев, когда выпал орёл, на $10\%$ больше, чем кол-во случаев, когда выпала решка. При каком $N$ мы можем сказать, что монета «нечестная» (орёл и решка выпадают с разной вероятностью)?


## ✅Решение:

Рассмотрим выборку наблюдений $x_1, ... , x_N$, которую мы получили в результате $N$ подбрасываний монеты.  
Пусть $x_i = 1$ - выпал орёл, а $x_i = 0$ - выпала решка.  
$cnt_1 = \sum_{i=1}^N x_i$ - кол-во случаев, когда выпал орёл.  
$cnt_0 = N - cnt_1$ - кол-во случаев, когда выпала решка.  

Необходимо проверить гипотезы:  
$H_0$: монета "честная".  
$H_1$: монета "нечестная".

$x_i$ описывается случайной величиной $\xi \sim \text{Bernoulli}(p)$. Тогда число орлов $cnt_1$ описывается случайной величиной $\eta$ равной числу успехов в $N$ независимых испытаниях Бернулли, то есть $cnt_1 \sim \text{Bin}(N, p)$. Следовательно $\mathbb{E}\eta = Np$.

Зафиксируем уровень статистической значимости $\alpha = 0.05$.

### 1️⃣ Критерий Стьюдента

Критерий Стьюдента проверяет гипотезу о равенстве матожиданий. В данном случае так как математическое ожидание случайной величины $\xi \sim \text{Bernoulli}(p)$ равно $p$ то можно переформулировать гипотезы для применения критерия Стьюдента:

Пусть $x^N = x_1, ... , x_N$ — реализация выборки $\xi^N$, $\xi \sim Bernoulli(p)$.  

$H_0$: $p = 0.5$    
$H_1$: $p \neq 0.5$

  
Пусть $\bar x$ - выборочное среднее $x_1, ... , x_N$.  
$\hat{\sigma} = \sqrt{\frac{\sum_{i=1}^{n}\left(x_i - \bar x\right)^2}{n-1}}$ - корень из их выборочной дисперсии.  
Тогда если $t = \frac{\bar x - \mu}{\frac{\hat\sigma}{\sqrt n}}$, то  
$t \sim \mathcal{St}\,(\text{df} = n - 1)$  
  
В данной задаче имеем двустороннюю альтернативную гипотезу, поэтому достигаемый уровень значимости ***p_value*** будем считать двусторонний.

Отметим, что критерий Стьюдента может ввести себя плохо в двух случаях: 

1) Есть выбросы в выборке (в нашем случае случайная величина принимает значения только $0$ и $1$, поэтому все ОК)  

2) Выборка настолько маленького размера, что её средне выборочное не имеет нормального распределения. (в нашем случае мы имеем дело с Бернулевскими случайными величинами, сумма которых ведет себя нормально уже при небольших $N$)

Поэтому в данной задаче корректно использовать критерий Стьюдента.
 

In [34]:
import numpy as np
from scipy.stats import t # распределение Стьюдента

def get_p_value_two_sided(t_statistic, n):
    """
    По t-статистике возвращает достигаемый уровень значимости
    p_value - двусторонний, так как альтернативная гипотеза была двусторонней.
    """
    if t_statistic > 0: # right-sided         
        one_sided_p = 1 - t(df=n - 1).cdf(t_statistic)
    else: # left-sided
        one_sided_p = t(df=n - 1).cdf(t_statistic)
    return min(one_sided_p * 2, 1)


def my_t_test(n, alpha):
    """
    T-test для данной задачи. Можно было бы воспользоваться реализованными
    методами из статистических библиотек, но для наглядности посчитаем
    t-статистику самостоятельно.
    Возвращает True - если верна альтернативная гипотеза H1.
    Возвращает False - если верна нулевая гипотеза H0.
    """
    cnt1 = int(0.55 * n)
    cnt0 = n - cnt1
    x = [1] * cnt1 + [0] * cnt0 # орлов на 10% больше, чем решек
    x_sample = np.array(x)
    n = x_sample.size
    sample_mean = np.mean(x_sample) # выборочное среднее
    estimated_sigma = np.std(x_sample, dtype=np.float64, ddof=1) # выборочное СКО
    estimated_sem = estimated_sigma / n ** 0.5 # SEM = standard error of the mean
    t_statistic = (sample_mean - 0.5) / estimated_sem
    # if H0 is true, t has distribution of Student(n - 1)
    p_value = get_p_value_two_sided(t_statistic, n)
    if p_value <= alpha:
        return True # принимаем H1
    else:
        return False # не отвергаем H0

In [36]:
for i in range(10, 500):
    if my_t_test(i, 0.05):
        print(i)

391
393
395
400
402
404
406
408
410
411
412
413
414
415
416
417
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499


Видим, что первые значения $N$, при которых нулевая гипотеза отвергается идут не подряд. Это связано с тем, что возникает ошибка при округлении числа решек и орлов. Однако при $N >= 410$ дальнейшие значения идут подряд. Поэтому для надежности, исключим шум, связанный с проблемами округления и будем считать, что при $N >= 410$ можно сказать, что монета нечестная.

### 2️⃣ Критерий $\chi^2$

В качестве бонуса приведу решение с помощью использования критерия хи-квадрат.  
Критерий хи-квадрат для матрицы сопряженности проверяет гипотезу о том, что группы связанные с каждым измерением независимы (или, что тоже самое, наблюдения имеют одно и то же распределение).  


In [29]:
from scipy import stats

def test_chi(n):
    alpha = 0.05  # фиксируем уровень cтатистической значимости
    x0 = 0.55 * n # число орлов
    x1 = 0.45 * n # число решек
    
    tau1, p_value, _, _ = stats.chi2_contingency(
    observed=[[x0, x1], [0.5 * n, 0.5 * n]], correction=False)
    
    return p_value

In [39]:
for i in range(20, 1000):
    if (test_chi(i) <= 0.05):
        print(i)
        break

767


Так, здесь мы получили ответ гораздо больший. Это можно объяснить тем, что критерий Стьюдента обладает большей мощностью, чем критерий хи-квдарат (в котором гипотеза более хитрая). Так что следует считать, что последнее слово за критерием Стьюдента.

### 👨🏻‍⚖️ Ответ:

При $N >= 410$ (если принять в расчет ошибки округления).